In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig ,BitsAndBytesConfig
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("/mnt/data1/shared/model/falcon-40b-instruct", trust_remote_code=True, padding=False)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant= True,
    bnb_4bit_quant_type="nf4"
)

#  /mnt/data1/shared/model/falcon-40b-instruct
#  tiiuae/falcon-7b-instruct

model = AutoModelForCausalLM.from_pretrained(
    "/mnt/data1/shared/model/falcon-40b-instruct",
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
    trust_remote_code=True
).eval()

generation_config = GenerationConfig(
    max_new_tokens=256,
    top_p=1,
    top_k=10,
    repetition_penalty=1
)
generation_config.eos_token_id = generation_config.pad_token_id = tokenizer.eos_token_id

/mnt/data1/miniconda3/envs/falcon/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /mnt/data1/miniconda3/envs/falcon/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /mnt/data1/miniconda3/envs/falcon/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/mnt/data1/miniconda3/envs/falcon/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /mnt/data1/miniconda3/envs/falcon did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
Loading checkpoint shards: 100%|██████████| 9/9 [03:14<00:00, 21.56s/it]


In [6]:
system_prompt ='''
Suppose you are a math teacher
'''

input_text = '''
2 3 5 7 11 according to left number sequence, what is the next number and why?
'''

input_ids = tokenizer.encode(system_prompt, input_text, return_tensors="pt", padding=False, add_special_tokens=False).to(device)

output = model.generate(
    input_ids,
    generation_config=generation_config
)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)
generated_text = output_text[len(system_prompt):].strip()
print(generated_text)

2 3 5 7 11 according to left number sequence, what is the next number and why?
The next number in the sequence is 13, because it is the next prime number after 11. The sequence is based on the Fibonacci sequence, which is a sequence of numbers where each number is the sum of the two preceding numbers. In this case, the sequence is based on the leftmost digit of each number, so the sequence goes 2, 3, 5, 7, 11, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 128849, 2147483647, etc.
